In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import time
import subprocess
from shapely.geometry import MultiPolygon
import pickle
# from all_config import *
import os
# import gem_tracker_maps.trackers.save_file_to_s3 as save_file_to_s3
from save_file_to_s3 import *

In [ ]:
full_country_list = [
    "Algeria", "Angola", "Benin", "Botswana", "British Indian Ocean Territory", "Burkina Faso", 
    "Burundi", "Cabo Verde", "Cameroon", "Central African Republic", "Chad", "Comoros", "DR Congo", 
    "Republic of the Congo", "Côte d'Ivoire", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", 
    "Eswatini", "Ethiopia", "French Southern Territories", "Gabon", "The Gambia", "Ghana", "Guinea", 
    "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", 
    "Mauritius", "Mayotte", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Réunion", "Rwanda", 
    "Saint Helena, Ascension, and Tristan da Cunha", "Sao Tome and Principe", "Senegal", "Seychelles", 
    "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo", "Tunisia", 
    "Uganda", "Western Sahara", "Zambia", "Zimbabwe",
    
    "Anguilla", "Antigua and Barbuda", "Argentina", "Aruba", "Bahamas", "Barbados", "Belize", "Bermuda", 
    "Bolivia", "Bonaire, Sint Eustatius, and Saba", "Bouvet Island", "Brazil", "Canada", "Cayman Islands", 
    "Chile", "Colombia", "Costa Rica", "Cuba", "Curaçao", "Dominica", "Dominican Republic", "Ecuador", 
    "El Salvador", "Falkland Islands", "French Guiana", "Greenland", "Grenada", "Guadeloupe", "Guatemala", 
    "Guyana", "Haiti", "Honduras", "Jamaica", "Martinique", "Mexico", "Montserrat", "Nicaragua", "Panama", 
    "Paraguay", "Peru", "Puerto Rico", "Saint Barthélemy", "Saint Kitts and Nevis", "Saint Lucia", 
    "Saint Martin (French part)", "Saint Pierre and Miquelon", "Saint Vincent and the Grenadines", 
    "Sint Maarten (Dutch part)", "South Georgia and the South Sandwich Islands", "Suriname", 
    "Trinidad and Tobago", "Turks and Caicos Islands", "United States", "Uruguay", "Venezuela", 
    "Virgin Islands (British)", "Virgin Islands (U.S.)",
    
    "Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Cambodia", "China", 
    "Cyprus", "Georgia", "Hong Kong", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", 
    "Kazakhstan", "North Korea", "South Korea", "Kuwait", "Kyrgyzstan", "Laos", "Lebanon", "Macao", 
    "Malaysia", "Maldives", "Mongolia", "Myanmar", "Nepal", "Oman", "Pakistan", "Palestine", "Philippines", 
    "Qatar", "Saudi Arabia", "Singapore", "Sri Lanka", "Syria", "Taiwan", "Tajikistan", "Thailand", 
    "Timor-Leste", "Türkiye", "Turkmenistan", "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen",
    
    "Åland Islands", "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", 
    "Bulgaria", "Croatia", "Czech Republic", "Denmark", "Estonia", "Faroe Islands", "Finland", "France", 
    "Germany", "Gibraltar", "Greece", "Guernsey", "Holy See", "Hungary", "Iceland", "Ireland", "Isle of Man", 
    "Italy", "Jersey", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "North Macedonia", 
    "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "Norway", "Poland", "Portugal", "Romania", 
    "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Svalbard and Jan Mayen", "Sweden", 
    "Switzerland", "Ukraine", "United Kingdom",
    
    "American Samoa", "Australia", "Christmas Island", "Cocos (Keeling) Islands", "Cook Islands", "Fiji", 
    "French Polynesia", "Guam", "Heard Island and McDonald Islands", "Kiribati", "Marshall Islands", 
    "Micronesia", "Nauru", "New Caledonia", "New Zealand", "Niue", "Norfolk Island", 
    "Northern Mariana Islands", "Palau", "Papua New Guinea", "Pitcairn", "Samoa", "Solomon Islands", 
    "Tokelau", "Tonga", "Tuvalu", "United States Minor Outlying Islands", "Vanuatu", "Wallis and Futuna"
]


In [ ]:
internal_map_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/internal/compilation_output/internal_2025-04-15.geojson'

In [ ]:
gdf = gpd.read_file(internal_map_file)

In [ ]:
gdf.info()


In [ ]:

def check_countries_official_multiarea(df,col_country_name, col_wiki):
    df = df.copy()
    problem_units_weird_country = []
    # for country in col remove ; at end
    # df_country_list = df[col_country_name].unique().tolist()
    df_country_wiki = df[[col_country_name, col_wiki]] # df wiht just these two columns
    
    # now a list of countries separated by ;
    # df_country_wiki[col_country_name] = df_country_wiki[col_country_name].apply(lambda x: x.replace(';', ''))
     
    # df_country_wiki.explode('col_country_name') explode # TODO keep the original country area1 etc so can match up subnat
    
    for row in df_country_wiki.index:
        # for item in df_country_wiki.loc[row,col_country_name]:
        #     item = item.replace(';', '')
        if df_country_wiki.loc[row,col_country_name] in full_country_list:
            pass
        else:
            problem_units_weird_country.append((df.loc[row,col_wiki], df.loc[row,col_country_name]))

    df = df.reset_index()
    print(problem_units_weird_country)
    return problem_units_weird_country


In [ ]:
# get unit-id, coords, all location data, and tracker 
# see by tracker and by country if there are set number of subnat options
subnat_stan = gdf[['subnat', 'url', 'geometry', 'id', 'subnat2', 'region2', 'area2', 'subnat', 'region', 'areas', 'tracker-display']]

In [ ]:
problem_units_weird_country = check_countries_official_multiarea(subnat_stan,'areas', 'url')
newlist = []
for item in problem_units_weird_country:
    areas = item[1]
    area_list = areas.split(';') # TODO turn into a list and check each one! 
    for area in area_list:
        if area in full_country_list:
            # remove area from list
            # problem_units_weird_country - item
            pass
        else:
            newlist.append(item)


In [ ]:
newlist

In [ ]:
len(newlist)

In [ ]:
subnat_stan

In [ ]:
# class RegionObject:
#     def __init__(self,
#                  name="",
#                  countryareas=[], # list of objects
#                  geometries=gpd.GeoDataFrame()):
#         self.name = name
#         self.countryareas = countryareas
#         self.geometries=geometries
        
#         def set_countryareas(self):
            
            
    

class CountryAreaObject:
    def __init__(self,
                 name="",
                 subnats=[],
                 geometries=gpd.GeoDataFrame()):
        self.name=name
        self.subnats=subnats
        self.geometries=geometries
        
            
        

In [ ]:
# for areas in subnat_stan find set list of subnat options
# subnat_stan_areas = subnat_stan.groupby('areas', as_index=False)['subnat']
unique_areas = set(subnat_stan['areas'].to_list())
all_country_objs = []
# create a region object, with area, subnat, coordinates
def create_area_object(df, area):
    
    area_obj = CountryAreaObject(
        name=area,
        subnats = df[df['areas']==area]['subnat'],   # get unique
        geometries=df[df['areas']==area]['geometry'] # get unique
    )
    
    return area_obj

for area in unique_areas:
    area_obj = create_area_object(subnat_stan, area)
    all_country_objs.append(area_obj)
    print(area)
    print(area_obj.subnats)
    
    
         

In [ ]:
# for row in area_obj.geometries.index:
#     print(row)
# print(area_obj.geometries)      
 
for obj in all_country_objs:
    lst_subnat = []
    if 'Argentina;' in obj.name:
        print(type(obj.subnats))
        print(obj.subnats.columns)
        for row in obj.subnats.index:
            subnat = obj.subnats.loc[row,'subnat']

            if subnat in lst_subnat:
                pass
            else:
                lst_subnat.append(subnat)


In [ ]:
# file = 'local_pkl/about_df_dict_by_map_2025-02-07.pkl'
filegoit =  'GOIT/compilation_output/goit_2025-04-08.geojson'
# file_goget_json = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/goget/compilation_output/Oil & Gas Extraction-map-file-2025-02-20.csv'

In [4]:
# https://drive.google.com/file/d/1wy53Z56YeIad4Tod46tGcOmduxBPLZ7r/view?usp=drive_link can we access via this link?

ggit_lng_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson'
ggit_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-Gas-Pipelines-2024-12 DATA TEAM COPY.geojson' #'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-Gas-Pipelines-2023-12 copy.geojson'

ggit_lng_eu_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-EGT-Terminals-2025-02 DATA TEAM COPY.geojson'
ggit_eu_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02 DATA TEAM COPY.geojson'

goit_geojson = '/Users/gem-tah/Downloads/GEM-GOIT-Oil-NGL-Pipelines-2025-03.geojson'

list_of_geojsons = [ggit_lng_geojson, ggit_geojson, ggit_lng_eu_geojson, ggit_eu_geojson, goit_geojson]


for filepath in list_of_geojsons:
    gdf = gpd.read_file(filepath)
    # Ensure geometry is properly handled before saving as parquet
    if 'geometry' in gdf.columns:
        if not isinstance(gdf, gpd.GeoDataFrame):
            print("Converting DataFrame to GeoDataFrame...")
            gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs="EPSG:4326")
        
        # Convert geometry to WKT format for saving as Parquet
        gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)
        df = gdf
    else:
        print(f'Was not a gdf anyway oddly check in on that \n{filepath}')
        df = gdf    
    
    for col in df.columns:
        # check if mixed dtype
        if df[col].apply(type).nunique() > 1:
            # if so, convert it to string
            df[col] = df[col].fillna('').astype(str)
        
    
    print(df.shape)
    # newfilepath = '/'.join(filepath.split('/')[:-1])
    newfilepath = filepath.replace('.geojson', '.parquet').replace(' ', '_')
    print(f'{newfilepath}')
    df.to_parquet(path = f'{newfilepath}', index=False)
    # save_file_to_s3.saves3(f'{filepath}') # also save original
    saves3(newfilepath)

/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_23656/2757802324.py:23: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)
/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_23656/2757802324.py:33: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].fillna('').astype(str)


(1874, 52)
/Users/gem-tah/Downloads/GEM-GOIT-Oil-NGL-Pipelines-2025-03.parquet
upload: ../../../../../Downloads/GEM-GOIT-Oil-NGL-Pipelines-2025-03.parquet to s3://publicgemdata/latest/GEM-GOIT-Oil-NGL-Pipelines-2025-03.parquet



In [ ]:
# save most recent download to s3


In [ ]:
# list_of_links = ['https://docs.google.com/spreadsheets/d/1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0/', 'https://drive.google.com/file/d/1X6tpYmgDKUWZzSYGlz0LcCeE-DqGeQcp/view?usp=drive_link', 
#                  'https://docs.google.com/spreadsheets/d/1aZiKzbHfFXLSEFCeKwstvL1-DV98ErXDhyCx00abRgQ/', 'https://docs.google.com/spreadsheets/d/1dqawuLhgFzYI9GWRU9bwsDY1SQHjzxifZb7u1T8OXqA/',
#                  'https://docs.google.com/spreadsheets/d/1zUyj7t4FO0M9fc82wpG93fZmisGar0Syq1l5ZGyyxEM/', 'https://docs.google.com/spreadsheets/d/1TxoQ17GUDvRbQ82r90eexbRNHfPfi-50wm9lBih2eYc/',
#                  'https://docs.google.com/spreadsheets/d/1hB2NuXMMqCJD94muoe6-A9BCi4v-7hquScej44_QWas/', 'https://docs.google.com/spreadsheets/d/1crtLnGzOS6wLYweIb84gLQpMV4A49qD5fty6WBTdLt0/',
#                  'https://docs.google.com/spreadsheets/d/1jUbsItBq_CVXy0dUxCP7APHfstDL85BaykCXYURagX4/', 'https://docs.google.com/spreadsheets/d/1bUTPRTmIqu7URVECm66NGr_DF7v32UUOzBaBf1dNawk/',
#                  'https://docs.google.com/spreadsheets/d/1x-IxdiqMJ_v-M_45ASFdg55Gl3Hsx3vb-7wjt8qCq5k/', 'https://docs.google.com/spreadsheets/d/1cy0A7k0GoB2b_b2wesObcAvts0GbXa7go7FVE_bHpBc/',
#                  'https://docs.google.com/spreadsheets/d/1XxAvcobKZ9XRKvHaSHDb4JbOstqhvrVWkU5Wu8JKhgE/', 'https://docs.google.com/spreadsheets/d/1gOOwjN551r6TllpBpfxtwNqF1yfbLbXdCFwACvKYeI4/',
#                  'https://docs.google.com/spreadsheets/d/10Clsb8auutXy4iBm6c7HywZjSbNKUiho9NHzZmC2-Bs/', 'https://docs.google.com/spreadsheets/d/1cqLe0xOx7FLpfaGZhtVnV-bhuKgaczHPtXzCRIgy_RQ/',
#                  'https://docs.google.com/spreadsheets/d/1zCcO9LwVmiQZBOGqlwKGYgZ5lAnBKJ91hjwWcbUw5nQ/', 'https://docs.google.com/spreadsheets/d/1zCcO9LwVmiQZBOGqlwKGYgZ5lAnBKJ91hjwWcbUw5nQ/',
#                  'https://docs.google.com/spreadsheets/d/16MoNKcz68GVwv6ztZ6p0vSNlL6EmPd633DZ7V0xG_Ho/']


# for link in list_of_links:
    
# save_file_to_s3.googlelinktoparquet(link)

In [ ]:
df = gpd.read_file(filegoit)


In [ ]:
# df.to_parquet(f"goit-map-2025-03.parquet", index=False)  # partition_cols=["country/area"],
# file = 'goit-map-2025-03.parquet'
file = filegoit

do_command_s3 = (
            f'export BUCKETEER_BUCKET_NAME=publicgemdata && '
            f'aws s3 cp {file} s3://$BUCKETEER_BUCKET_NAME/latest/ '
            f'--endpoint-url https://nyc3.digitaloceanspaces.com --acl public-read')

        # Execute the terminal command to pull down file from digital ocean
process = subprocess.run(do_command_s3, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print(process.stdout.decode('utf-8'))
if process.stderr:
    print(process.stderr.decode('utf-8'))


In [ ]:
set(df['status-legend'].to_list())

In [ ]:
# df = pickle.load(file)
# df = gpd.read_file(file_goget_json)

In [ ]:

# country_list = set(df['country-list'].to_list())
# print(country_list)

countries = set(df['country/area'].apply(lambda x: x.split('-')).apply(lambda x: '; '.join(x)).to_list())
print(sorted(countries))

# missing = country_list - countries


# print(missing)

In [ ]:
current_countries = {
  "Africa": [
    "Algeria",
    "Angola",
    "Angola-Republic of the Congo",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "Côte d'Ivoire",
    "Djibouti",
    "DR Congo",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Republic of the Congo",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Senegal-Mauritania",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ],
  "Americas": [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bermuda",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Canada",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Greenland",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Puerto Rico",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Pierre and Miquelon",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "United States",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)",
    "Virgin Islands (U.S.)"
  ],
  "Asia": [
    "Afghanistan",
    "Armenia",
    "Azerbaijan",
    "Azerbaijan-Turkmenistan",
    "Bahrain",
    "Bangladesh",
    "Bhutan",
    "Brunei",
    "Cambodia",
    "China",
    "China-Japan",
    "Cyprus",
    "Georgia",
    "Hong Kong",
    "India",
    "Indonesia",
    "Iran",
    "Iran-Iraq",
    "Iraq",
    "Israel",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kuwait",
    "Kuwait-Saudi Arabia",
    "Kuwait-Saudi Arabia-Iran",
    "Kyrgyzstan",
    "Laos",
    "Lebanon",
    "Macao",
    "Malaysia",
    "Maldives",
    "Mongolia",
    "Myanmar",
    "Nepal",
    "North Korea",
    "Oman",
    "Pakistan",
    "Palestine",
    "Philippines",
    "Qatar",
    "Russia-Kazakhstan",
    "Saudi Arabia",
    "Saudi Arabia-Bahrain",
    "Saudi Arabia-Iran",
    "Singapore",
    "South China Sea",
    "South Korea",
    "Sri Lanka",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Thailand",
    "Thailand-Malaysia",
    "Timor-Leste",
    "Türkiye",
    "Turkmenistan",
    "United Arab Emirates",
    "United Arab Emirates-Iran",
    "Uzbekistan",
    "Vietnam",
    "Vietnam-Malaysia",
    "Yemen"
  ],
  "Europe": [
    "Åland Islands",
    "Albania",
    "Andorra",
    "Austria",
    "Belarus",
    "Belgium",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Faroe Islands",
    "Finland",
    "France",
    "Germany",
    "Gibraltar",
    "Greece",
    "Guernsey",
    "Holy See",
    "Hungary",
    "Iceland",
    "Ireland",
    "Isle of Man",
    "Italy",
    "Jersey",
    "Kosovo",
    "Latvia",
    "Liechtenstein",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Moldova",
    "Monaco",
    "Montenegro",
    "Netherlands",
    "North Macedonia",
    "Norway",
    "Poland",
    "Portugal",
    "Romania",
    "Russia",
    "Russia-Kazakhstan",
    "San Marino",
    "Serbia",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Svalbard and Jan Mayen",
    "Sweden",
    "Switzerland",
    "Ukraine",
    "United Kingdom"
  ],
  "Oceania": [
    "American Samoa",
    "Australia",
    "Christmas Island",
    "Cocos (Keeling) Islands",
    "Cook Islands",
    "Fiji",
    "French Polynesia",
    "Guam",
    "Heard Island and McDonald Islands",
    "Kiribati",
    "Marshall Islands",
    "Micronesia",
    "Nauru",
    "New Caledonia",
    "New Zealand",
    "Niue",
    "Norfolk Island",
    "Northern Mariana Islands",
    "Palau",
    "Papua New Guinea",
    "Pitcairn",
    "Samoa",
    "Solomon Islands",
    "Timor Gap",
    "Tokelau",
    "Tonga",
    "Tuvalu",
    "United States Minor Outlying Islands",
    "Vanuatu",
    "Venezuela-Trinidad and Tobago",
    "Wallis and Futuna"
  ]
}


current_countries_list = []

for v in current_countries.values():
    current_countries_list.extend(v)

print(current_countries_list)

print(current_countries_list)

In [ ]:
countries_list = list(countries)
countries_list = [c.replace('; ', '-') for c in countries_list]

missing_countries_areas = set(countries_list) - set(current_countries_list)

In [ ]:
missing_countries_areas

In [ ]:
# add missing_countries_areas to current_countries 


compare all regional outputs and source data

In [ ]:
folder_path_pkl = f'{tracker_folder_path}local_pkl'


for folder in ['LATAM', 'africa-energy', 'asia']:
    
    file_path = f'{folder}/config.js'
    if folder == 'africa-energy':
        folder = 'africa'
        
    with open(file_path, 'r') as file:
        jscontent = file.read()
        # print(type(jscontent)) # string
        # print(jscontent)
        # make df from json file 
        if 'geojson:' in jscontent:
            second_part = jscontent.split('json: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]
            folder_full = tracker_folder_path + str(folder)


            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')
            current_file_line = str(current_file_line)
            print(current_file_line)
            current_file_line_csv = current_file_line.replace('geojson', 'csv')

            full_path = os.path.join(folder_full,current_file_line)

            # df = pd.read_json(full_path)
            # change to csv version that is / should be saved next to it b/c easier to read
            full_path_csv = os.path.join(folder_full,current_file_line_csv)
            try:
                df = pd.read_csv(full_path_csv)
            except FileNotFoundError:
                current_file_line_xlsx = current_file_line.replace('json', 'xlsx')
                full_path_xlsx = os.path.join(folder_full,current_file_line_xlsx)
                print(full_path_xlsx)
                try: 
                    df = pd.read_excel(full_path_xlsx)
                    
                except FileNotFoundError:
                    print(f'Check the folder {folder_full} seemingly no alternatives to json tried csv and xlsx')
            finally:
                # print(df.head())
                print(f'Length df: {len(df)}')

                
        
        elif 'json' in jscontent:
            second_part = jscontent.split('json: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]

            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')

            folder_full = tracker_folder_path + str(folder)

            current_file_line = str(current_file_line)
            print(current_file_line)
            current_file_line_csv = current_file_line.replace('geojson', 'csv')

            full_path = os.path.join(folder_full,current_file_line)

            # df = pd.read_json(full_path)
            # change to csv version that is / should be saved next to it b/c easier to read
            full_path_csv = os.path.join(folder_full,current_file_line_csv)
            try:
                df = pd.read_csv(full_path_csv)
            except FileNotFoundError:
                current_file_line_xlsx = current_file_line.replace('geojson', 'xlsx')
                full_path_xlsx = os.path.join(folder_full,current_file_line_xlsx)
                print(full_path_xlsx)
                try: 
                    df = pd.read_excel(full_path_xlsx)
                except FileNotFoundError:
                    print(f'Check the folder {folder_full} seemingly no alternatives to json tried csv and xlsx')
            finally:
                # print(df.head())
                # print(len(df))
                print(f'Length df: {len(df)}')

            
        elif 'csv:' in jscontent:
            second_part = jscontent.split('csv: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]

            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')


            folder_full = tracker_folder_path + str(folder)
            full_path = os.path.join(folder_full,current_file_line)
            df = pd.read_csv(full_path)
            print(f'Length df: len(df)')

            
        else:
            print(jscontent)

        print(date_pkl)
        # filter for tracker to update 
        print(folder)
        print(len(df))
        df_filt = df[df["tracker-acro"]=='GOGET']
        print(len(df_filt))

            
    

totals:
AGT: Map - 474, table - 479 (potentially Thailand-Malaysia, China-Japan, Vietnam-Malaysia missing)
LATAM - map - 1195, table - 1200 (missing- Venezuela-Trinidad and Tobago?)
AET:Map - 665, table 667 (Senegal-Mauritania, Angola-Republic of the Congo)

In [ ]:
# pull in source
# filter for region and fuel
folder_path_pkl = f'{tracker_folder_path}local_pkl'
file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'

full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
df_pkl = pd.read_pickle(full_path_pkl)
f'Length of source after being filtered: {len(df_pkl)}'


Check the summary colab to see how he filters region and fuel, must be that. 
And pull in europe to this slew of functions so have those pkl files too

From scotts summary code 
Regional Tracker Summary Tables
https://colab.research.google.com/drive/1c1Y7MHV6f8lEAeb8l0nw1B_d1ETmwVRb#scrollTo=e4yXRRIkch6v 

In [ ]:
# AET covers "The Africa Energy Tracker covers the subregions of Northern Africa and Sub-Saharan Africa" https://globalenergymonitor.org/projects/africa-energy-tracker/methodology/
# Define African Sites
african_countries = [
    "Algeria", "Angola", "Angola-Republic of the Congo", "Benin", "Botswana",
    "British Indian Ocean Territory", "Burkina Faso", "Burundi", "Cabo Verde",
    "Cameroon", "Central African Republic", "Chad", "Comoros", "Côte d'Ivoire",
    "Djibouti", "DR Congo", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini",
    "Ethiopia", "French Southern Territories", "Gabon", "Ghana", "Guinea",
    "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar",
    "Malawi", "Mali", "Mauritania", "Mauritius", "Mayotte", "Morocco",
    "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo",
    "Réunion", "Rwanda", "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe", "Senegal", "Senegal-Mauritania", "Seychelles",
    "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan",
    "Tanzania", "The Gambia", "Togo", "Tunisia", "Uganda", "Western Sahara",
    "Zambia", "Zimbabwe"
]


# Define Asian Countries https://globalenergymonitor.org/projects/asia-gas-tracker/methodology/
asia_countries = [
    # South Asia
    "Afghanistan", "Bangladesh", "Bhutan", "India", "Iran",
    "Maldives", "Nepal", "Pakistan", "Sri Lanka",

    # Southeast Asia
    "Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia",
    "Myanmar", "Philippines", "Singapore", "Thailand",
    "Timor-Leste", "Vietnam", "Thailand-Malaysia",  "Vietnam-Malaysia",

    # East Asia
    "China", "China-Japan", "Hong Kong", "Japan", "Macao",
    "Mongolia", "North Korea", "South Korea", "Taiwan",

    # Multinational or Maritime Areas
    "South China Sea"
]

# Define European Countries and Territories https://globalenergymonitor.org/projects/europe-gas-tracker/methodology/
europe_countries = [
    # Northern Europe
    "Åland Islands", "Denmark", "Estonia", "Faroe Islands", "Finland",
    "Iceland", "Ireland", "Isle of Man", "Latvia", "Lithuania",
    "Norway", "Sweden", "United Kingdom", "Guernsey", "Jersey", "Svalbard and Jan Mayen",

    # Western Europe
    "Austria", "Belgium", "France", "Germany", "Liechtenstein",
    "Luxembourg", "Monaco", "Netherlands", "Switzerland",

    # Eastern Europe
    "Belarus", "Czech Republic", "Hungary", "Moldova", "Poland",
    "Romania", "Slovakia", "Ukraine",

    # Southern Europe
    "Albania", "Andorra", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Cyprus", "Gibraltar", "Greece", "Italy", "Kosovo", "Malta",
    "Montenegro", "North Macedonia", "Portugal", "San Marino",
    "Serbia", "Slovenia", "Spain", "Türkiye",

    # Special Cases
    "Israel"  # list in the methodology
]

# Define Latin American and Caribbean Countries and Territories - see "Data request for the new year"
latin_america_caribbean = [
    # Caribbean
    "Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados",
    "Belize", "Cayman Islands", "Cuba", "Curaçao", "Dominica",
    "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica",
    "Martinique", "Montserrat",  "Saint Barthélemy",
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin (French part)",
    "Saint Vincent and the Grenadines", "Sint Maarten (Dutch part)",
    "Trinidad and Tobago", "Turks and Caicos Islands", "Virgin Islands (British)",
   # "Virgin Islands (U.S.)", "Puerto Rico", -  gregor excludes

    # Central America
    "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico",
    "Nicaragua", "Panama",

    # South America
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador",
    "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname",
    "Uruguay", "Venezuela",

    # Special Cases
    "Bonaire, Sint Eustatius, and Saba", "Bouvet Island",
    "Falkland Islands", "South Georgia and the South Sandwich Islands",
    "Venezuela-Trinidad and Tobago"
]

In [ ]:
# map countries 


africa_countries_map = [
    "Algeria",
    "Angola",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "DR Congo",
    "Republic of the Congo",
    "Côte d'Ivoire",
    "Djibouti",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ]


asia_countries_map = [ 
        "China",
        "Hong Kong",
        "Japan",
        "Macao",
        "Mongolia",
        "North Korea",
        "South Korea",
        "Taiwan",
        "Brunei",
        "Cambodia",
        "Indonesia",
        "Laos",
        "Malaysia",
        "Myanmar",
        "Philippines",
        "Singapore",
        "Thailand",
        "Timor-Leste",
        "Vietnam",
        "Afghanistan",
        "Iran",
        "Bangladesh",
        "Bhutan",
        "India",
        "Maldives",
        "Nepal",
        "Pakistan",
        "Sri Lanka"]

# european_union_countries = [
#     'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
#     'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 
#     'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 
#     'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 
#     'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 
#     'Spain', 'Sweden',
# ]
# other_europe_countries = [
#     'Albania', 'Andorra', 'Belarus', 'Bosnia and Herzegovina', 'Holy See', 'Iceland',
#     'Liechtenstein', 'Moldova', 'Monaco', 'Montenegro', 'North Macedonia', 
#     'Norway', 'San Marino', 'Serbia', 'Switzerland', 'Türkiye', 'Ukraine', 
#     'United Kingdom', 'Israel',
# ]

europe_countries_map = [
    'Åland Islands', 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 
    'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czech Republic', 
    'Denmark', 'Estonia', 'Faroe Islands', 'Finland', 'France', 'Germany', 
    'Gibraltar', 'Greece', 'Guernsey', 'Holy See', 'Hungary', 'Iceland', 
    'Ireland', 'Isle of Man', 'Italy', 'Jersey', 'Kosovo', 'Latvia', 
    'Liechtenstein', 'Lithuania', 'Luxembourg', 'North Macedonia', 'Malta', 
    'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 
    'Portugal', 'Romania', 'Israel', 'San Marino', 'Serbia', 'Slovakia', 
    'Slovenia', 'Spain', 'Svalbard and Jan Mayen', 'Sweden', 'Switzerland', 
    'Ukraine', 'United Kingdom', 'Cyprus', 'Türkiye'
]



latam_countries_map = [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)"]

In [ ]:
#compare differenc with subtract a set

missing_eu = set(europe_countries) - set(europe_countries_map) 
# missing_eu = set(europe_countries_map) - set(europe_countries) 

print(missing_eu)

missing_asia = set(asia_countries) - set(asia_countries_map)
# missing_asia = set(asia_countries_map) - set(asia_countries)

print(missing_asia)

missing_africa = set(african_countries) - set(africa_countries_map)
# missing_africa = set(africa_countries_map) - set(african_countries)

print(missing_africa)

missing_latam = set(latin_america_caribbean) - set(latam_countries_map)
# missing_latam = set(latam_countries_map) - set(latin_america_caribbean)

print(missing_latam)



In [ ]:
# map 949 goget eu
# scott 964.0 goget eu

GOGET architecture exploring

In [ ]:
# 7k projects goget
goget_file = '/Users/gem-tah/Downloads/GOGET oil & gas extraction sites (main data set) - Main data.csv'

goget_df = pd.read_csv(goget_file)
print(set(goget_df['Project or complex'].to_list()))

# print(goget_df['Unit type'])
print(goget_df.columns)



In [ ]:
projects = goget_df[goget_df['Unit type']=='project']
not_projects = goget_df[goget_df['Unit type']!='project']
print(len(projects))
print(len(not_projects))

not_in_project = goget_df[goget_df['Project or complex'].isna()]
print(len(not_in_project))

print(len(goget_df))

# are those that complexes that have no data but have fields associated with them get published in the official data download

# find the rows that are connected to a project,

         
# how many units are in a project 

In [ ]:
# we want a list of all the goget units that do not agree with its project
#  Unit type = field, project, complex 
# Project or complex = name of corresponding project 

In [ ]:
hier_top = ['project', 'complex'] # block
hier_middle = ['field', 'area', 'asset', 'pool', 'phase']
hier_middle_new = ['field', 'area', 'asset', 'pool', 'phase', 'sub-basin', 'concession', 'basin']
# hier_low = [''] # 'sub-basin, concession, basin

cols_hier_top = ['Owner 1p', 'Owner 1f', 'Owner 1 %p', 'Owner 1 %f', 'Owner 2p', 'Owner 2f', 'Owner 2 %p', 'Owner 2 %f', 'Owner 3p', 'Owner 3f', 'Owner 3 %p', 'Owner 3 %f', 'Owner 4p', 'Owner 4f', 'Owner 4 %p', 'Owner 4 %f', 'Owner 5p', 'Owner 5f', 'Owner 5 %p', 'Owner 5 %f', 'Owner % sump', 'Owner % sumf'] # FID, start up, owner, onshore/offshore
cols_hier_middle = ['Latitudep', 'Latitudef', 'Longitudep', 'Longitudef', 'Location accuracyp', 'Location accuracyf', 'Operatorp', 'Operatorf'] # discover, operator, lat, long, basin, formation, block
cols_agree_needed = ['Country/Areap', 'Country/Areaf', 'Subnational unit (province, state)p', 'Subnational unit (province, state)f', 'Statusp', 'Statusf']
basic_cols = ['project ID (inferred)','project name (inferred)','Unit IDp','Unit IDf', 'Unit Namep','Unit Namef', 'Project or complexp', 'Project or complexf']

set(goget_df['Unit type'].to_list())
questions = '1.)what to do with 480 that arent clearly top or mid from that schema?'

In [ ]:
# field project groupings for existing hierarchy data reconciliation
# a field is in a grouping if its Unit type = field and its Project or complex is not blank
# a project is in a grouping if its Unit type = project or complex and in a pivot table it has fields
# find situations where field and project do not correspond, match on name 

# groupings = [] # tuples of project first, then list of fields
field_gr = goget_df.copy()
print(len(field_gr))
# field_gr = field_gr[field_gr['Unit type'].isin(hier_middle)]
field_gr = field_gr[field_gr['Unit type'].isin(hier_middle_new)]

field_gr = field_gr[field_gr['Project or complex'].notna()] # 14177 without this filter 3540 with it
field_gr['Project or complex'] = field_gr['Project or complex'].apply(lambda x:x.strip())
print(len(field_gr))
field_proj_names = field_gr['Project or complex'].to_list()
print(field_proj_names)

proj_gr = goget_df.copy()
print(len(proj_gr))
proj_gr = proj_gr[proj_gr['Unit type'].isin(hier_top)]
proj_gr['Unit Name'] = proj_gr['Unit Name'].apply(lambda x:x.strip())
print(len(proj_gr))
proj_names = proj_gr['Unit Name'].to_list()
print(proj_names)

no_overlap = set(field_proj_names) - set(proj_names) 
print(no_overlap)
print(len(goget_df)-len(proj_gr)-len(field_gr)) # 480 neither field or project 


In [ ]:
# merge project rows with field rows 
print(cols_agree_needed + basic_cols)
groupings = proj_gr.merge(field_gr, left_on='Unit Name', right_on='Project or complex', how='inner', suffixes=('p', 'f'))
# rename Unit IDp to be project id for easier pivoting
# groupings.rename(columns={'Unit IDp': 'project ID (inferred)', 'Unit Namep': 'project name (inferred)'}, inplace=True)
groupings['project ID (inferred)'] = groupings['Unit IDp']
groupings['project name (inferred)'] = groupings['Unit Namep']
# len(groupings)
# print(groupings)


cols_rec_check = groupings[cols_agree_needed + basic_cols]
cols_rec_check_proj = groupings[cols_hier_top + basic_cols]
cols_rec_check_field = groupings[cols_hier_middle + basic_cols]

all_groupings = groupings[basic_cols + cols_agree_needed + cols_hier_top + cols_hier_middle]
# all_groupings.to_csv(f'all_groupings_goget 2025-03-10.csv')


In [ ]:

# groupings
# cols_rec_check
# cols_rec_check_proj
# cols_rec_check_field
# pivot 
# group on project ID and find inconsistencies
# pivot_cols = basic_cols.pop(2)
# del basic_cols[2] # project ID (inferred)
# pivot_cols = [basic_cols.pop(basic_cols.index('project ID (inferred)'))]
# pivot_cols = cols_agree_needed + cols_hier_top + cols_hier_middle
# all_groupings = all_groupings.groupby(['project ID (inferred)','project name (inferred)'])
# all_groupings = all_groupings.pivot(index='project ID (inferred)', columns = pivot_cols)
# print(all_groupings)

# a file showing side by side cols that do not match between project and field, but so that each field is lined up along project
# cols_hier_top = ['Owner 1', 'Owner 1 %', 'Owner 2', 'Owner 2 %', 'Owner 3', 'Owner 3 %', 'Owner 4', 'Owner 4 %', 'Owner 5', 'Owner 5 %', 'Owner % sum'] # FID, start up, owner, onshore/offshore
# cols_hier_middle = ['Latitude', 'Longitude', 'Location accuracy', 'Operator'] # discover, operator, lat, long, basin, formation, block
# cols_agree_needed = ['Country/Area', 'Subnational unit (province, state)', 'Status']
to_reconcile = {}
# loop through and see if values match, if they do not they get added to a tuple according to the project id
all_cols = cols_hier_top + cols_hier_middle + cols_agree_needed
# strip last character and de duplicate
all_cols = [col[:-1] for col in all_cols]
all_cols = list(set(all_cols))
all_groupings = all_groupings.fillna('')
for row in all_groupings.index:
    proj_id = all_groupings.loc[row, 'Unit IDp']
    proj_name = all_groupings.loc[row, 'Unit Namep']
    unit_name = all_groupings.loc[row, 'Unit Namef']
    unit_id = all_groupings.loc[row, 'Unit IDf']
    for col in all_cols:
        
        field_val = all_groupings.loc[row,f'{col}f']
        proj_val = all_groupings.loc[row,f'{col}p']
        if field_val != proj_val:
            if field_val == '' or proj_val == '':
                continue
            else:
                to_reconcile[(proj_id, proj_name, unit_id, unit_name, col)] = [field_val, proj_val]
                
to_reconcile_df = pd.DataFrame.from_dict(to_reconcile, orient='index')    
to_reconcile_df.to_excel(f'to_reconcile_goget_2025-03-11.xlsx', index=True)


In [ ]:


# print(f'Ask Scott: {questions}')

